## Assignment 2
### Problem 6.
For this problem, you will use each of the three models you’ve constructed in problems 2–4 to
evaluate the probability of a toy corpus of sentences (containing just the first two sentences of the
Brown corpus) found at: toy_corpus.txt

The sentences are contained in this corpus in the same format as the other corpora you’ve been
using so far: one sentence per line and words separated by a space. As before, you’ll need to remove
the end-of-line character.

In [50]:
# Problem 2,  wouter's version
##
import nltk
import numpy as np
from nltk.corpus import brown
from collections import Counter
from nltk import pos_tag

vocab = open("brown_vocab_100.txt")

#load the indices dictionary
word_index_dict = {}
for i, line in enumerate(vocab):
    #import part 1 code to build dictionary
    word_index_dict[line.rstrip()] = i

f = open("brown_100.txt")

counts = np.zeros(len(word_index_dict)) #initialize counts to a zero vector

#iterate through file and update counts
for sent in f:
    sent_list = sent.lower().split()

    for word in sent_list:
        ind = word_index_dict[word]
        counts[ind] += 1

f.close()

#normalize and writeout counts. 

#print(counts) edited for space
probs = counts / np.sum(counts)
print(probs[0])
print(probs[-1])

0.0004051863857374392
0.003646677471636953


#### *Beginning of Problem 6*

In [91]:
#open and rea dthe toy_corpus.txt
toy_corpus_file = open("toy_corpus.txt", "r")
toy_corpus_text = toy_corpus_file.read()
toy_corpus_file.close()
sentences = toy_corpus_text.split('\n')

#open to write the probabilities into the new txt file
output_file = open("unigram_eval.txt", "w")

#iterate thru sentences and calculate joint prob of each one
for sentence in sentences:
    words = sentence.split()
    #initialize sentence prob
    sentprob = 1
    #computes the joint prob per word in sentence
    for word in words:
        #finds index of word
        index = word_index_dict.get(word.lower(), None)
        if index is not None:
            #finds prob of word
            wordprob = probs[index]
            #updated sentence prob by * with word prob
            sentprob *= wordprob
    #To calculate the perplexity, first calculate the length of the sentence in words (be sure to
    #include the end-of-sentence word)   
    sent_len =len(words) - words.count("</s>")  + 1
    #calculate perplexity
    perplexity = 1/(pow(sentprob, 1.0/sent_len))    
    
    #added the perplexity check to unigram_eval.txt (instead of joint prob, to check joint, have it write
    #output_file.write(str(sentprob) + '\n') instead
    output_file.write(str(perplexity) + '\n')

#close file
output_file.close()

#verify jopint pro b is 4.84008387782
print('Joint probability of the 2nd sentence: ', sentprob)

print('sent_len: ', sent_len)
print('sentprob: ', sentprob)
#cheeck perplexity of 2nd sentence is 153
print('perplexity of the 2nd sentence: ', perplexity)

Joint probability of the 2nd sentence:  4.840083877820268e-99
sent_len:  45
sentprob:  4.840083877820268e-99
perplexity of the 2nd sentence:  153.03157461392973
